In [13]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import os

# 데이터 로드
file_path = "C:\\Users\\NSE\\Downloads\\진짜_최종_키워드 결합.csv"
output_dir = "C:\\Users\\NSE\\Downloads"
output_file = os.path.join(output_dir, "YYYYMM 키워드 결합 릿지.xlsx")

df = pd.read_csv(file_path)

# 데이터 타입 확인 및 수치형 컬럼 변환
numeric_cols = ['Website purchases', 'Return on ad spend (ROAS)', 'Website checkouts initiated', 'Amount spent', 'CTR (link click-through rate)', 'C/R']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 나이대와 성별 인코딩
df_age = pd.get_dummies(df['Age'], prefix='나이대', drop_first=True)
df_gender = pd.get_dummies(df['Gender'], prefix='성별', drop_first=False)

# 열 인덱스를 기준으로 인덱싱 (17번째 열부터 끝까지)
df_keywords = df.iloc[:, 17:]

# 연도와 월 별로 그룹화하여 릿지 회귀 분석 수행
results = []

for period, group in df.groupby('YYYYMM'):
    # 독립 변수와 종속 변수를 결합
    X = pd.concat([group['Amount spent'], df_keywords.loc[group.index], df_age.loc[group.index], df_gender.loc[group.index]], axis=1)
    y = group['Website purchases']

    # 데이터 타입 확인 및 변환
    X = X.apply(pd.to_numeric, errors='coerce')
    y = pd.to_numeric(y, errors='coerce')
    X = X.astype(float)

    # 결측값 제거
    X = X.dropna()
    y = y.loc[X.index]

    # 상수항 추가
    X = sm.add_constant(X)

    # Min-Max 스케일링
    scaler = MinMaxScaler()
    X['Amount spent'] = scaler.fit_transform(X[['Amount spent']])

    # 모델 적합
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X, y)

    # 예측 값 계산
    y_pred = ridge_model.predict(X)

    # 모델 평가
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    # 모델 결과 저장
    result = {
        '변수': ['Mean Squared Error', 'R^2 Score', 'Intercept'] + list(X.columns),
        '값': [mse, r2, ridge_model.intercept_] + list(ridge_model.coef_)
    }

    results.append((str(period), pd.DataFrame(result)))

# 엑셀 파일로 저장
try:
    with pd.ExcelWriter(output_file) as writer:
        for period, result_df in results:
            sheet_name = period
            result_df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"File saved successfully at {output_file}")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")


File saved successfully at C:\Users\NSE\Downloads\YYYYMM 키워드 결합 릿지.xlsx


In [14]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import os

# 데이터 로드
file_path = "C:\\Users\\NSE\\Downloads\\진짜_최종_키워드 결합.csv"
output_file = os.path.join("C:\\Users\\NSE\\Downloads", "202301_202404 키워드 결합 릿지.xlsx")

df = pd.read_csv(file_path)

# 데이터 타입 확인 및 수치형 컬럼 변환
numeric_cols = ['Website purchases', 'Return on ad spend (ROAS)', 'Website checkouts initiated', 'Amount spent', 'CTR (link click-through rate)', 'C/R']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 나이대와 성별 인코딩
df_age = pd.get_dummies(df['Age'], prefix='나이대', drop_first=True)
df_gender = pd.get_dummies(df['Gender'], prefix='성별', drop_first=False)

# 열 인덱스를 기준으로 인덱싱 (17번째 열부터 끝까지)
df_keywords = df.iloc[:, 17:]

# 연도와 월을 분리하여 추가
df['Year'] = df['YYYYMM'].astype(str).str[:4].astype(int)
df['Month'] = df['YYYYMM'].astype(str).str[4:].astype(int)

# 2023년 1월부터 2024년 4월까지 각각의 월 데이터를 처리
start_year = 2023
end_year = 2024
start_month = 1
end_month = 4

results = []

for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        # 2024년 4월까지만 처리
        if year == end_year and month > end_month:
            break
        # 2023년 1월부터 시작
        if year == start_year and month < start_month:
            continue

        # 해당 월의 데이터 추출
        df_month = df[(df['Year'] == year) & (df['Month'] == month)]
        
        if df_month.empty:
            continue

        # 해당 월의 데이터에 대한 독립 변수와 종속 변수 설정
        X_month = pd.concat([df_month['Amount spent'], df_keywords.loc[df_month.index], df_age.loc[df_month.index], df_gender.loc[df_month.index]], axis=1)
        y_month = df_month['Website purchases']
        
        # 데이터 타입 확인 및 변환
        X_month = X_month.apply(pd.to_numeric, errors='coerce')
        y_month = pd.to_numeric(y_month, errors='coerce')
        X_month = X_month.astype(float)
        
        # 결측값 제거
        X_month = X_month.dropna()
        y_month = y_month.loc[X_month.index]
        
        # 상수항 추가
        X_month = sm.add_constant(X_month)
        
        # Min-Max 스케일링
        scaler = MinMaxScaler()
        X_month['Amount spent'] = scaler.fit_transform(X_month[['Amount spent']])
        
        # 릿지 회귀 모델 및 파라미터 설정
        ridge = Ridge()
        parameters = {'alpha': [0.1, 1.0, 10.0, 100.0]}
        
        # 교차 검증을 사용한 Grid Search
        clf = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
        clf.fit(X_month, y_month)
        
        # 최적의 파라미터 출력
        best_alpha = clf.best_params_['alpha']
        print(f"Best alpha for {year}-{month:02}: {best_alpha}")
        
        # 최적의 alpha 값을 사용하여 다시 릿지 회귀 분석
        ridge_model_month = Ridge(alpha=best_alpha)
        ridge_model_month.fit(X_month, y_month)
        y_pred_month = ridge_model_month.predict(X_month)
        mse_month = mean_squared_error(y_month, y_pred_month)
        r2_month = r2_score(y_month, y_pred_month)
        
        # 해당 월의 데이터 결과 저장
        result_month = {
            '변수': ['Mean Squared Error', 'R^2 Score', 'Intercept'] + list(X_month.columns),
            '값': [mse_month, r2_month, ridge_model_month.intercept_] + list(ridge_model_month.coef_)
        }
        df_result_month = pd.DataFrame(result_month)
        
        # 결과를 리스트에 추가
        results.append((f'{year}_{month:02}', df_result_month))

# 엑셀 파일로 저장
try:
    with pd.ExcelWriter(output_file) as writer:
        for sheet_name, result_df in results:
            result_df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"File saved successfully at {output_file}")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")


Best alpha for 2023-01: 0.1
Best alpha for 2023-02: 0.1
Best alpha for 2023-03: 0.1
Best alpha for 2023-04: 0.1
Best alpha for 2023-05: 0.1
Best alpha for 2023-06: 0.1
Best alpha for 2023-07: 0.1
Best alpha for 2023-08: 0.1
Best alpha for 2023-09: 0.1
Best alpha for 2023-10: 0.1
Best alpha for 2023-11: 0.1
Best alpha for 2023-12: 0.1
Best alpha for 2024-01: 0.1
Best alpha for 2024-02: 1.0
Best alpha for 2024-03: 0.1
Best alpha for 2024-04: 0.1
File saved successfully at C:\Users\NSE\Downloads\202301_202404 키워드 결합 릿지.xlsx


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os

# 데이터 로드
file_path = "C:\\Users\\NSE\\Downloads\\진짜_최종_키워드 결합.csv"
output_file = os.path.join("C:\\Users\\NSE\\Downloads", "개선된 릿지 회귀.xlsx")

df = pd.read_csv(file_path)

# 데이터 타입 확인 및 수치형 컬럼 변환
numeric_cols = ['Website purchases', 'Return on ad spend (ROAS)', 'Website checkouts initiated', 'Amount spent', 'CTR (link click-through rate)', 'C/R']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 나이대와 성별 인코딩
df_age = pd.get_dummies(df['Age'], prefix='나이대', drop_first=True)
df_gender = pd.get_dummies(df['Gender'], prefix='성별', drop_first=False)

# 열 인덱스를 기준으로 인덱싱 (17번째 열부터 끝까지)
df_keywords = df.iloc[:, 17:]

# 연도와 월을 분리하여 추가
df['Year'] = df['YYYYMM'].astype(str).str[:4].astype(int)
df['Month'] = df['YYYYMM'].astype(str).str[4:].astype(int)

# 개선이 필요한 월 목록
target_months = [(2023, 6), (2023, 10), (2023, 11), (2024, 2), (2024, 3), (2024, 4)]

results = []

for year, month in target_months:
    df_month = df[(df['Year'] == year) & (df['Month'] == month)]
    
    if df_month.empty:
        continue

    # 독립 변수와 종속 변수 설정
    X_month = pd.concat([df_month['Amount spent'], df_keywords.loc[df_month.index], df_age.loc[df_month.index], df_gender.loc[df_month.index]], axis=1)
    y_month = df_month['Website purchases']
    
    # 데이터 타입 확인 및 변환
    X_month = X_month.apply(pd.to_numeric, errors='coerce')
    y_month = pd.to_numeric(y_month, errors='coerce')
    X_month = X_month.astype(float)
    
    # 결측값 제거
    X_month = X_month.dropna()
    y_month = y_month.loc[X_month.index]
    
    # 상수항 추가
    X_month = sm.add_constant(X_month)

    # 데이터가 비어있지 않은지 확인
    if X_month.empty:
        print(f"Data for {year}-{month:02} is empty after outlier removal.")
        continue

    # 데이터 변환 (PowerTransformer)
    pt = PowerTransformer()
    X_month = pd.DataFrame(pt.fit_transform(X_month), columns=X_month.columns)
    
    # 표준화
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_month)

    # 데이터 분할 (훈련/검증)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_month, test_size=0.2, random_state=42)
    
    # 릿지 회귀 모델 및 파라미터 설정
    ridge = Ridge()
    parameters = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
    
    # 교차 검증을 사용한 Grid Search
    clf = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
    clf.fit(X_train, y_train)
    
    # 최적의 파라미터 출력
    best_alpha = clf.best_params_['alpha']
    print(f"Best alpha for {year}-{month:02}: {best_alpha}")
    
    # 최적의 alpha 값을 사용하여 다시 릿지 회귀 분석
    ridge_model = Ridge(alpha=best_alpha)
    ridge_model.fit(X_train, y_train)
    
    # 예측 값 계산 및 평가
    y_pred_train = ridge_model.predict(X_train)
    y_pred_test = ridge_model.predict(X_test)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    # 모델 결과 저장
    result = {
        '변수': ['MSE Train', 'R^2 Train', 'MSE Test', 'R^2 Test', 'Intercept'] + list(X_month.columns),
        '값': [mse_train, r2_train, mse_test, r2_test, ridge_model.intercept_] + list(ridge_model.coef_)
    }

    results.append((f'{year}_{month:02}', pd.DataFrame(result)))

# 엑셀 파일로 저장
try:
    with pd.ExcelWriter(output_file) as writer:
        for sheet_name, result_df in results:
            result_df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"File saved successfully at {output_file}")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")


Best alpha for 2023-06: 10.0
Best alpha for 2023-10: 10.0
Best alpha for 2023-11: 100.0
Best alpha for 2024-02: 100.0
Best alpha for 2024-03: 100.0
Best alpha for 2024-04: 100.0
File saved successfully at C:\Users\NSE\Downloads\개선된 릿지 회귀.xlsx
